In [ ]:
import pandas
import pandas as pd
import numpy
import numpy as np
import os
from IPython.display import Markdown

In [ ]:
import ipywidgets as widgets
from IPython.display import display


In [ ]:
color_print = lambda x, y: display(Markdown(f'# <span style="color:{y}">{x}</span>'))
from joblib import Memory
memory = Memory("cachedir")
pandas.read_gbq = memory.cache(pandas.read_gbq)

In [ ]:
conditions = {
#     "Squamous cell carcinoma of skin": 4111921,
#     "Arthritis":4291025,
    "Asthma":317009,
    "Afib":313217,
    "CKD":46271022,
    "Heart disease":321588,
    "Hyperlipidemia": 432867,
    "HTN": 316866,
    "Hyperthyroidism": 4142479,
    "Hypothyroidism": 140673,
    "Obesity": 433736,
    "Diabetes mellitus_type_2": 201826,
#     "Myasthenia Gravis": 76685,
    "Chronic obstructive lung disease": 255573,
#     "Rheumatoid arthritis": 80809,
    "Gastroesophageal reflux disease": 318800,
#     "Ulcerative colitis": 81893,
#     "Irritable bowel_disease": 75576,
#     "Kaposi Sarcoma": 434584,
#     "Prurigo Nodularis": 4272156,
#     "Generalized anxiety disorder": 434613,
#     "Bipolar disorder": 436665,
#     "Insomnia": 436962,
#     "Schizophrenia": 435783,
#     "Posttraumatic stress disorder": 436676,
#     "Anorexia nervosa": 436675,
#     "Bulimia nervosa": 438407,
#     "Alcoholism": 4218106,
    "Kaposis Sarcoma": 434584,
    "HIV": 439727,
#     "Alopecia areata": 141933,
#     "Atopic dermatitis": 133834,
#     "Psoriasis": 140168,
#     "Rosacea":136773,
#     "Squamous Cell Carcinoma": 4300118,
#     "Seborrheic keratosis":40568467,
#     "Verruca vulgaris":140641,
#     "Vitiligo": 138502,
#     "Pemphigus": 135338,
#     "Cutaneous lupus": 4324123,
#     "Systemic lupus erythematosus": 257628,
#     "Localized scleroderma": 441928,
#     "Lichen": 141094,
    "Generalized anxiety_disorder": 434613,
    "Major depressive_disorder": 4152280,
#     "Bipolar disorder": 436665,
#     "Insomnia":436962,
#     "Schizophrenia": 435783,
#     "Posttraumatic stress disorder": 436676,
#     "Anorexia nervosa": 436675,
#     "Bulimia nervosa": 438407,
#     "Alcoholism": 4218106,
}

inflammatory_conditions = {
#     "Acne": 141095, 
#     "Hidradenitis Suppurativa": 4241223,
#     "Atopic Dermatitis": 133834,
#     "Psoriasis": 140168, 
#     "Lichen Planus": 132703,
#     "Rosacea": 136773,
#     "Seborrheic Dermatitis": 137053,
#     "Folliculitis": 4029295, 
#     "Irritant Contact Dermatitis": 4004352,
}

autoimmune_conditions = {
    
#     "Behcets Syndrome": 436642, 
#     "Dermatomyositis": 80182, 
#     "Lichen Planus": 132703, 
#     "Lupus Erythematosus": 255891, 
#     "Localized Scleroderma": 441928, 
#     "Dermatitis Herpetiformis": 140487, 
#     "Pemphigoid": 139899, 
#     "Vitiligo": 138502, 
#     "Pemphigus": 135338, 
#     "Vasculitis": 4137275, 
#     "Epidermolysis Bullosa": 4246290,  
#     "Alopecia Areata": 141933
    # "Pyoderma gangrenosum": 133283,
#     'Vitiligo': 138502,
#     "Alopecia areata": 141933,
#     "Cutaneous lupus erythematosus": 4324123,
#     "Systemic lupus erythematosus": 257628,
#     "Localized scleroderma": 441928,
    # "Sjogren's Syndrome": 254443,
#     "Lichen": 141094,
#     "Psoriasis": 140168,
#     "Atopic dermatitis": 133834,
}

conditions = conditions | autoimmune_conditions | inflammatory_conditions

for i, j in autoimmune_conditions.items():
    if i not in conditions.keys():
        print(f"Key '{i}' from 'autoimmune_conditions' not found in 'conditions'. Make sure you add '{i}' to conditions")

In [ ]:
survey_question_dict = {
    'race': 1586140,
    'gender': 1585838,
    'deaf': 903573,
    'blind': 903574,
    'difficulty_concentrating': 903575,
    'difficulty_walking_climbing': 903576,
    'difficulty_dressing_bathing': 903577,
    'difficulty_errands_alone': 903578,
    'low_income': 1585375,
    'education_level': 1585940,
    'smoking': 1585857,
    'alcohol': 1586198,
    'ecig': 1586166,
    'marijuana': 1585650,
    'sexual_orientation': 1585899,
    'employment_status': 1585952,
    'has_health_insurance': 1585386,
    'health_insurance_type': 1585389,
    'gender_identity': 1585838,
    'Cocaine': 1585656, 
    'Sedative': 1585680, 
    'Street_Opioid': 1585692, 
    'Prescription_Opioid': 1585698, 
    'Prescription_Stimulant': 903058, 
    'Cigar': 1586174, 
    'Hookah': 1586182, 
    'Smokeless_Tobacco': 1586190
}

In [ ]:
os.environ["WORKSPACE_CDR"]
project_id, dataset_id = os.environ["WORKSPACE_CDR"].split(".")
project_id

In [ ]:
# # This query represents dataset "Full Cohort" for domain "person" and was generated for All of Us Registered Tier Dataset v7
dataset_person_sql = """
    SELECT
        person.person_id,
        person.birth_datetime as date_of_birth,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth ,
        cb_search_person.has_ehr_data as has_ehr_data,
        cb_search_person.age_at_cdr as age
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person
            ON person.person_id = cb_search_person.person_id
    WHERE cb_search_person.has_ehr_data = 1"""

dataset_person_df = pandas.read_gbq(
    dataset_person_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

# dataset_person_df
# dataset_person_df = dataset_79490832_person_df
dataset_person_df.head()

In [ ]:
os.environ["WORKSPACE_CDR"]


In [ ]:
# This query represents dataset "Full Cohort" for domain "survey" and was generated for All of Us Registered Tier Dataset v7
#     answer.survey,
#     answer.question,
#     answer.answer,
dataset_survey_questions_sql = """
SELECT DISTINCT
    answer.survey,
    answer.question,
    answer.question_concept_id,
    answer.answer,
    answer.answer_concept_id,
    cb_search_person.has_ehr_data as has_ehr_data
FROM
    `""" + os.environ["WORKSPACE_CDR"] + f""".ds_survey` answer
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + f""".cb_search_person` cb_search_person
        ON answer.person_id = cb_search_person.person_id
WHERE 
    cb_search_person.has_ehr_data = 1"""

dataset_survey_questions_df = pandas.read_gbq(
    dataset_survey_questions_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

# dataset_survey_df = dataset_79490832_survey_df

dataset_survey_questions_df.head(5)

In [ ]:
# dataset_survey_questions_df[dataset_survey_questions_df['survey'] == "Lifestyle"]['question'].unique()

In [ ]:

def multi_checkbox_widget(options_dict):
    """ Widget with a search field and lots of checkboxes """
    search_widget = widgets.Text()
    output_widget = widgets.Output()
    options = [x for x in options_dict.values()]
    options_layout = widgets.Layout(
        overflow='auto',
        border='1px solid black',
        width='300px',
        height='300px',
        flex_flow='column',
        display='flex'
    )
    
    #selected_widget = wid.Box(children=[options[0]])
    options_widget = widgets.VBox(options, layout=options_layout)
    #left_widget = wid.VBox(search_widget, selected_widget)
    multi_select = widgets.VBox([search_widget, options_widget])

    @output_widget.capture()
    def on_checkbox_change(change):
        
        selected_recipe = change["owner"].description
        #print(options_widget.children)
        #selected_item = wid.Button(description = change["new"])
        #selected_widget.children = [] #selected_widget.children + [selected_item]
        options_widget.children = sorted([x for x in options_widget.children], key = lambda x: x.value, reverse = True)
        
    for checkbox in options:
        checkbox.observe(on_checkbox_change, names="value")

    # Wire the search field to the checkboxes
    @output_widget.capture()
    def on_text_change(change):
        search_input = change['new']
        if search_input == '':
            # Reset search field
            new_options = sorted(options, key = lambda x: x.value, reverse = True)
        else:
            # Filter by search field using difflib.
            #close_matches = difflib.get_close_matches(search_input, list(options_dict.keys()), cutoff=0.0)
            close_matches = [x for x in list(options_dict.keys()) if str.lower(search_input.strip('')) in str.lower(x)]
            new_options = sorted(
                [x for x in options if x.description in close_matches], 
                key = lambda x: x.value, reverse = True
            ) #[options_dict[x] for x in close_matches]
        options_widget.children = new_options

    search_widget.observe(on_text_change, names='value')
    return multi_select, output_widget

In [ ]:
# selected_survey_questions_list = dataset_survey_questions_df[dataset_survey_questions_df['survey'] == survey_prompt.value]['question'].unique().tolist()
def get_survey_selector_ui(survey_name, options_dict):
    

    def f(**args):

        results = [key for key, value in args.items() if value]
        display(results)

    ui, output_widget = multi_checkbox_widget(options_dict)
#     display(output_widget)
    out = widgets.interactive_output(f, options_dict)
    ui_out_hbox = widgets.HBox([ui, out])
    return ui_out_hbox, options_dict



In [ ]:
survey_selection_dict = {
    i: {
        x: widgets.Checkbox(
            description=x, 
            value=False,
            style={"description_width":"0px"}
        ) for x in dataset_survey_questions_df[dataset_survey_questions_df['survey'] == i]['question'].unique().tolist()
    } for i in dataset_survey_questions_df['survey'].unique().tolist()
} 
tab_contents = [i for i in survey_selection_dict.keys() if i in ["The Basics", "Lifestyle"]]

def generate_child(survey_name, options_dict):
    
    survey_ui_out, out_options_dict = get_survey_selector_ui(survey_name, options_dict)
    child = widgets.VBox([widgets.Label(survey_name), survey_ui_out])
    return child

children = [generate_child(name, survey_selection_dict[name]) for name in tab_contents]
tab = widgets.Tab()
tab.children = children
tab.titles = tab_contents
tab

In [ ]:
user_selected_survey_questions_nested = [[k for k, v in survey_selection_dict[i].items() if v.value] for i in survey_selection_dict.keys()]

In [ ]:
user_selected_survey_questions = [i for j in user_selected_survey_questions_nested for i in j]

In [ ]:
survey_question_selected_concept_ids = dataset_survey_questions_df[dataset_survey_questions_df['question'].isin(user_selected_survey_questions)]['question_concept_id'].unique()

In [ ]:
# survey_question_dict_set = set(survey_question_dict.values())
survey_question_dict_set = set(survey_question_selected_concept_ids)
survey_q_of_interest = set([1585899, 1585940, 1585952, 1585375, 1585386, 1585389, 1585838, 903573, 903574, 903575, 903576, 903577, 903578, 1586140, 1585857, 1586198, 1585650, 1586166, 1585357])
survey_q_of_interest_set = (survey_q_of_interest).union(survey_question_dict_set)
survey_q_of_interest = tuple(survey_q_of_interest_set)
survey_question_dict.values()
str(survey_q_of_interest)

In [ ]:
# This query represents dataset "Full Cohort" for domain "survey" and was generated for All of Us Registered Tier Dataset v7
dataset_survey_sql = """
SELECT
    answer.person_id,
    answer.survey_datetime,
    answer.survey,
    answer.question_concept_id,
    answer.question,
    answer.answer_concept_id,
    answer.answer,
    answer.survey_version_concept_id,
    answer.survey_version_name,
    cb_search_person.has_ehr_data as has_ehr_data
FROM
    `""" + os.environ["WORKSPACE_CDR"] + f""".ds_survey` answer
LEFT JOIN
    `""" + os.environ["WORKSPACE_CDR"] + f""".cb_search_person` cb_search_person
        ON answer.person_id = cb_search_person.person_id
WHERE 
    cb_search_person.has_ehr_data = 1 AND
    answer.question_concept_id IN {str(survey_q_of_interest)}"""

dataset_survey_df = pandas.read_gbq(
    dataset_survey_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

# dataset_survey_df = dataset_79490832_survey_df

dataset_survey_df.head(5)

In [ ]:
unique_questions = dataset_survey_df['question'].unique().tolist()

In [ ]:
unique_question_answers = {i[0]:i_df['answer'].unique().tolist() for i, i_df in dataset_survey_df[['question', 'answer']].groupby(['question'])}


In [ ]:
unique_question_answers

In [ ]:
unique_question_and_id = dataset_survey_df[['question', 'question_concept_id']].drop_duplicates()

In [ ]:
survey_question_dict = {i: j for i, j in zip(unique_question_and_id['question'], unique_question_and_id['question_concept_id'])}

In [ ]:
def get_unique_comorbidities():
    query = """
SELECT DISTINCT
  condition_occurrence.condition_concept_id AS snomed_code,
  concept.concept_name AS condition_name
FROM 
  `""" + os.environ["WORKSPACE_CDR"] + """.condition_occurrence` condition_occurrence
JOIN 
  `""" + os.environ["WORKSPACE_CDR"] + """.concept` concept 
ON 
  condition_occurrence.condition_concept_id = concept.concept_id
WHERE 
  concept.domain_id = 'Condition'
  AND concept.concept_class_id = 'Clinical Finding'
"""

    result = pd.read_gbq(
        query,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook"
    )

    return result
unique_comorbidities = get_unique_comorbidities()

In [ ]:
fix_condition_names = lambda _conditions_dict: {unique_comorbidities[unique_comorbidities["snomed_code"] == v]['condition_name'].values[0].replace("'", '').replace("(","").replace(")", ""):v for k,v in _conditions_dict.items()}

conditions = fix_condition_names(conditions)
autoimmune_conditions = fix_condition_names(autoimmune_conditions)
inflammatory_conditions = fix_condition_names(inflammatory_conditions)

In [ ]:
def get_common_comorbidities(target_c):
    query = f"""
WITH persons_with_target_condition AS (
  SELECT DISTINCT person_id
  FROM {os.environ["WORKSPACE_CDR"]}.condition_occurrence
  WHERE condition_concept_id = {target_c}
),
person_conditions AS (
  SELECT person_id, condition_concept_id
  FROM (
    SELECT DISTINCT person_id, condition_concept_id
    FROM {os.environ["WORKSPACE_CDR"]}.condition_occurrence
    WHERE person_id IN (SELECT person_id FROM persons_with_target_condition)
  ) AS distinct_conditions
)
SELECT condition_concept_id, COUNT(*) AS count
FROM person_conditions
GROUP BY condition_concept_id
ORDER BY count DESC;
"""

  

    query3 ="""WITH persons_with_target_condition AS (
  SELECT DISTINCT person_id
  FROM """ + os.environ["WORKSPACE_CDR"] + f""".condition_occurrence
  WHERE condition_concept_id = {target_c}
),
person_conditions AS (
  SELECT person_id, condition_concept_id
  FROM """ + os.environ["WORKSPACE_CDR"] + """.condition_occurrence
  WHERE person_id IN (SELECT person_id FROM persons_with_target_condition)
)
SELECT condition_concept_id, COUNT(*) AS count
FROM person_conditions
GROUP BY condition_concept_id
ORDER BY count DESC;"""

    result = pd.read_gbq(
        query,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook"
    )

    return result
# common_comorbidities = get_common_comorbidities(target_condition_concept_id)

In [ ]:
conditions_df_dict = {}
for condition, concept_id in [i for i in conditions.items() if i not in conditions_df_dict.keys()]:

    # This query represents dataset "Pemphigus survey" for domain "person" and was generated for All of Us Registered Tier Dataset v7
    _dataset_person_sql = f"""
        SELECT
            person.person_id,
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
        WHERE
            person.PERSON_ID IN (
                SELECT
                    distinct person_id  
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
                WHERE
                    cb_search_person.person_id IN (
                        SELECT
                            criteria.person_id 
                        FROM
                            (SELECT
                                DISTINCT person_id,
                                entry_date,
                                concept_id 
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                            WHERE
                                (
                                    concept_id IN (
                                        SELECT
                                            DISTINCT c.concept_id 
                                        FROM
                                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                                        JOIN
                                            (
                                                select
                                                    cast(cr.id as string) as id 
                                                FROM
                                                    `""" + os.environ["WORKSPACE_CDR"] + f""".cb_criteria` cr 
                                                WHERE
                                                    concept_id IN ({concept_id}) 
                                                    AND full_text LIKE '%_rank1]%'
                                            ) a 
                                                ON (
                                                    c.path LIKE CONCAT('%.',
                                                a.id,
                                                '.%') 
                                                OR c.path LIKE CONCAT('%.',
                                                a.id) 
                                                OR c.path LIKE CONCAT(a.id,
                                                '.%') 
                                                OR c.path = a.id) 
                                            WHERE
                                                is_standard = 1 
                                                AND is_selectable = 1
                                            ) 
                                            AND is_standard = 1 
                                    )
                                ) criteria 
                            ) )"""

    _dataset_person_df = pandas.read_gbq(
        _dataset_person_sql,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook")
    print(f"Condition = {condition}")
    print(f"ID = {concept_id}")
    conditions_df_dict[condition.replace(" ", "_").replace("(","").replace(")", "")] = _dataset_person_df
    display(_dataset_person_df.head(5))
    print("-"*50)

In [ ]:
# Column Combiner

import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

groups = {}

# container to hold all the rows
rows_container = widgets.VBox()

# output area to show status / current groups
status_output = widgets.Output()

def make_row():
    tags_input = widgets.TagsInput(
        value=[],
        allowed_tags=list(conditions_df_dict.keys()),
        allow_duplicates=False,
        layout=widgets.Layout(width="300px", min_width='300px')
    )
    logic_dropdown = widgets.Dropdown(
        options=['AND', 'OR'],
        value='OR',
        layout=widgets.Layout(width='100px', min_width='100px')
    )
    new_col_input = widgets.Text(
        placeholder='e.g. combined_col',
#         layout=widgets.Layout(width='150px', min_width='150px')
        style={'description_width': 'initial'}
    )
    run_button = widgets.Button(
        description='Add Group',
        layout=widgets.Layout(width='100px', min_width='100px'),
        icon='plus'
    )

    def on_click(b):
        name = new_col_input.value.strip()
        tags = list(tags_input.value)
        logic = logic_dropdown.value

        with status_output:
            clear_output()
            if not name:
                print("Please provide a new column name!")
                return
            if not tags:
                print("Please select at least one tag/condition!")
                return

            groups[name] = {'tags': tags, 'logic': logic}
            display(widgets.HTML(f"<b>Added group '{name}!'</b>"))
            print("\nCurrent groups:")
            for col, spec in groups.items():
                print(f" • {col}: {spec['logic']}({', '.join(spec['tags'])})")

        # append a fresh row and disable this button
        rows_container.children += (make_row(),)
        run_button.disabled = True

    run_button.on_click(on_click)

    return widgets.HBox(
        [tags_input, logic_dropdown, new_col_input, run_button],
        layout=widgets.Layout(
            display='flex',
            width='100%',
            justify_content='space-between',
            align_items='center',
            margin='5px 0'
        )
    )

# start with one row
rows_container.children = (make_row(),)

# single header row with equal-width columns
header = widgets.HBox(
    children=[
        widgets.HTML('<b>Conditions</b>'),
        widgets.HTML('<b>Logic</b>'),
        widgets.HTML('<b>Column Name</b>'),
        widgets.HTML('<b>Action</b>'),
    ],
    layout=widgets.Layout(
            display='flex',
            width='100%',
            justify_content='space-between',
            align_items='center',
            margin='5px 0'
    )
)

# display everything
display(widgets.VBox([header, rows_container, status_output]))

In [ ]:
def sanitize(inp_str):
    allowed_chars = {chr(i) for i in range(ord('A'), ord('z')+1) if chr(i).isalpha()}
    return ''.join([i if i in allowed_chars else "_" for i in inp_str])
#     return inp_str.replace(" ", "_").replace("(","").replace(")", "").replace(":","_").replace("'","_")

In [ ]:
sanitize("edfqwerg3451g34t3++__!4143ss")

# Define survey question and answer groups

In [ ]:
# #  ANNUAL_INCOME_Q = 1585375
# #     LOW_INCOME = [1585376, 1585377]
# #     HIGH_INCOME = [1585378 , 1585379, 1585380 , 1585381 , 1585382 , 1585383 , 1585384 ]



# survey_question_groups = {
#     'deaf': {
#         'question_concept_id': 903573,
#         'answer_groups': {
#             'Deaf: Yes': [903587],
#             'Deaf: No': [903503]
#         }
#     },
#     'blind': {
#         'question_concept_id': 903574,
#         'answer_groups': {
#             'Blind: Yes': [903504],
#             'Blind: No': [903597]
#         }
#     },
#     'difficulty_concentrating': {
#         'question_concept_id': 903575,
#         'answer_groups': {
#             'Difficulty Concentrating: Yes': [903599],
#             'Difficulty Concentrating: No': [903600]
#         }
#     },
#     'difficulty_walking_climbing': {
#         'question_concept_id': 903576,
#         'answer_groups': {
#             'Difficulty Walking Climbing: Yes': [903602],
#             'Difficulty Walking Climbing: No': [903603]
#         }
#     },
#     'difficulty_dressing_bathing': {
#         'question_concept_id': 903577,
#         'answer_groups': {
#             'Difficulty Dressing Bathing: Yes': [903605],
#             'Difficulty Dressing Bathing: No': [903606]
#         }
#     },
#     'difficulty_errands_alone': {
#         'question_concept_id': 903578,
#         'answer_groups': {
#             'Difficulty Errands Alone: Yes': [903608],
#             'Difficulty Errands Alone: No': [903609]
#         }
#     },
#     'low_income': {
#         'question_concept_id': 1585375,
#         'answer_groups': {
#             'Low Income: Yes': [1585376, 1585377],
#             'Low Income: No': [1585378 , 1585379, 1585380 , 1585381 , 1585382 , 1585383 , 1585384 ]
#         }
#     }
# }


In [ ]:
def add_field_to_df(survey_df, Q_ID, in_dataset_person_df, field_name, print_info=False):
    field_name_print = " ".join([i.capitalize() for i in field_name.split("_")])
    # main_df_with_disabilities = None
    main_df_with_employment = None
    new_field_df = survey_df[survey_df['question_concept_id'] == Q_ID][['person_id', 'answer']]

    new_field_df[field_name] = new_field_df['answer'].apply(lambda x: f"{field_name_print}: Unknown" if pd.isna(x) else x)
    
    main_df_with_new_field = in_dataset_person_df.merge(new_field_df[['person_id', field_name]], on='person_id', how='left')
    main_df_with_new_field[field_name] = main_df_with_new_field[field_name].apply(lambda x: f"{field_name_print} Unknown" if pd.isna(x) else x)
#     main_df_with_new_field[field_name] = main_df_with_new_field[field_name].astype(int)
    if print_info:
        print(main_df_with_new_field[field_name].unique())
        print((survey_df[survey_df['answer'].isin(main_df_with_new_field[field_name].unique())]['answer']).unique())
        
    return main_df_with_new_field

In [ ]:
def add_field_to_df_manage_duplicates(survey_df, Q_ID, field_name, in_dataset_person_df, print_info=False):
    field_name_print = " ".join([i.capitalize() for i in field_name.split("_")])
    # main_df_with_disabilities = None
    main_df_with_employment = None
    new_field_df = survey_df[survey_df['question_concept_id'] == Q_ID][['person_id', 'answer']]
    print("--1")
    new_field_df[field_name] = new_field_df['answer'].apply(lambda x: f"{field_name_print}: Unknown" if pd.isna(x) else x)
    print("--2")
    main_df_with_new_field = in_dataset_person_df.merge(new_field_df[['person_id', field_name]], on='person_id', how='left')
    main_df_with_new_field[field_name] = main_df_with_new_field[field_name].apply(lambda x: f"{field_name_print}: Unknown" if pd.isna(x) else x)
#     main_df_with_new_field[field_name] = main_df_with_new_field[field_name].astype(int)
#     print(f"main_df_with_new_field[field_name].unique() = {main_df_with_new_field[field_name].unique()}")
    
    print("--3")
    person_ids_with_multiple_resp = (main_df_with_new_field.groupby('person_id').count() > 1).any(axis=1).reset_index()
    person_ids_with_multiple_resp = person_ids_with_multiple_resp[person_ids_with_multiple_resp[0] == True]
    
    
#     print(f"person_ids_with_multiple_resp = {person_ids_with_multiple_resp}")
    main_df_with_new_field_without_duplicates = main_df_with_new_field.drop_duplicates(subset='person_id')
#     print(f"main_df_with_new_field_without_duplicates = {main_df_with_new_field_without_duplicates}")
#     print(f"main_df_with_new_field_without_duplicates['person_id'].isin(person_ids_with_multiple_resp['person_id']) \n = {main_df_with_new_field_without_duplicates['person_id'].isin(person_ids_with_multiple_resp['person_id'])}")
#     print((main_df_with_new_field_without_duplicates['person_id'].isin(person_ids_with_multiple_resp['person_id'])))
    print("--4")
    main_df_with_new_field_without_duplicates.loc[
        main_df_with_new_field_without_duplicates['person_id'].isin(person_ids_with_multiple_resp['person_id']), 
        field_name
        ] = f'{field_name_print}: Multiselect'
    
    print(f"person_ids_with_multiple_resp.shape = {person_ids_with_multiple_resp.shape}")

#     if print_info:
#         print(main_df_with_new_field[field_name].unique())
#         print((survey_df[survey_df['answer'].isin(main_df_with_new_field[field_name].unique())]['answer']).unique())
        
    return main_df_with_new_field_without_duplicates[['person_id', field_name]]

In [ ]:
survey_question_dict


In [ ]:
main_df_with = {}
for k, v in survey_question_dict.items():
    print(f"k = {k}")
    main_df_with[sanitize(k)] = add_field_to_df_manage_duplicates(dataset_survey_df, v, sanitize(k), dataset_person_df, True)

# main_df_with_disabilities['age_above_65'] = (main_df_with_disabilities['age'] > 65).astype(int)

# main_df_with['is_low_income'] = main_df_with_income
# main_df_with['has_disability'] = main_df_with_disabilities

In [ ]:
for k, v in conditions_df_dict.items():
    como_df = dataset_person_df[['person_id']].copy()
#     v[k.replace(" ", "_").replace("(","").replace(")", "")] = f"{k}: Yes"
    v[sanitize(k)] = f"{k}: Yes"
    como_df = como_df.merge(v, on='person_id', how="left")
    como_df = como_df.fillna(f"{k}: No")
    main_df_with[k] = como_df

In [ ]:
# autoimmune_conditions_df_dict = {k:v for k,v in conditions_df_dict.items() if k in autoimmune_conditions.keys()}    
# autoimmune_combined_df = dataset_person_df[['person_id']].copy()
# for k, v in autoimmune_conditions_df_dict.items():
#     auto_df = dataset_person_df[['person_id']].copy()
#     v[sanitize(k)] = True
#     auto_df = auto_df.merge(v, on='person_id', how="left")
#     auto_df = auto_df.fillna(False)
# #     print(k)
# #     print(auto_df)
#     autoimmune_combined_df[k] = auto_df[sanitize(k)]

In [ ]:
import numpy as np

# def sanitize(name):
#     return name.replace(" ", "_").replace("(", "").replace(")", "")
def sanitize(inp_str):
    allowed_chars = {chr(i) for i in range(ord('A'), ord('z')+1) if chr(i).isalpha()}
    return ''.join([i if i in allowed_chars else "_" for i in inp_str])






# Treatment Data
## Drug Categories and Concept Codes

# Combine into a DF

In [ ]:
main_df_with.keys()


In [ ]:
final_df = dataset_person_df.copy()
# final_df = final_df.merge(final_df, on='person_id', how='left')

for k, v in main_df_with.items():
    if k not in ['is_low_income', 'has_disability']:
        final_df = final_df.merge(main_df_with[k][['person_id', sanitize(k)]], on='person_id', how='left')
        print(final_df.shape)

In [ ]:
final_df.columns

In [ ]:
for group_name, spec in groups.items():
    tags, logic = spec['tags'], spec['logic']  # e.g. ['A','B'], 'OR'
    cols = [sanitize(t) for t in tags]

    # Build a boolean DataFrame
    yes_flags = final_df[cols].apply(lambda s: s.str.endswith(': Yes'))

    if logic == 'OR':
        mask = yes_flags.any(axis=1)
    else:  # AND
        mask = yes_flags.all(axis=1)

    final_df[sanitize(group_name)] = np.where(
        mask,
        f"{group_name}: Yes",
        f"{group_name}: No"
    )

# Pretty Print master_df
final_df.head(1)

In [ ]:
final_df.keys()


In [ ]:
ehr_df = final_df[final_df['has_ehr_data'] == 1]


In [ ]:
def get_factor_counts(in_df, in_factor, in_target_variable):
    _df = in_df[[in_factor, in_target_variable]].copy()
    _df['count'] = 1
    _df = _df.groupby([in_factor, in_target_variable]).count().reset_index()
    _df = _df.pivot(index=in_factor, columns=in_target_variable, values='count').reset_index()
    _df = _df.fillna(0)
    _df['total'] = _df.loc[0] + _df.loc[1]
    return _df

# get_factor_counts(ehr_df, 'gender', 'race')

In [ ]:
X = ehr_df


In [ ]:
def validate_groups(validate_factor, validate_groups):
    print(f"Validating {validate_factor} groups.", end="")
    for i in X[validate_factor].unique():
        print(".", end="")
        found=False
        for k, v in validate_groups.items():
            if i in v:
                found=True
                break
            else:
                continue
        if found:
            continue
        else:
            print()
            print(f"WARNING: '{validate_factor}': '{i}' was not found in the '{validate_factor}' group. Please make sure the group definitions are correct")
    print('-'*(35-len(X[validate_factor].unique())-len(validate_factor)), end='')
    print('\tDONE')

# Create Factor Groups

## Select columns that you would like to modify

In [ ]:
all_cols = ehr_df.columns.tolist()
unique_values_for_each_col = {
    i: ehr_df[i].unique().tolist() for i in all_cols
}
col_selection_state_dict = {i: False for i in all_cols}
col_selection_checkbox_dict = {i: widgets.Checkbox(
    value=col_selection_state_dict[i],
    description=i,
    disabled=False,
    indent=False
    )
    for i in all_cols}

In [ ]:
items = [col_selection_checkbox_dict[i] for i in all_cols]
modify_columns_box = widgets.GridBox(items, layout=widgets.Layout(grid_template_columns="repeat(3, 300px)"))

In [ ]:
modify_columns_box

In [ ]:
import ipywidgets as widgets
from IPython.display import display

def generate_inner_accordion(allowed_tags):
    # Helper to create a new row
    def factor_group_row(allowed_tags):
        factor_new_name_text_input = widgets.Text(
            value='',
            placeholder='Enter a name for this category',
            description='Category Name:',
            disabled=False,
    #         layout=widgets.Layout(width='300px'),
            style={'description_width': 'initial'}
        )
        available_tags = widgets.TagsInput(
            value=allowed_tags,
            allowed_tags=allowed_tags,
            allow_duplicates=False
            )
        return widgets.HBox([factor_new_name_text_input, available_tags]), factor_new_name_text_input, available_tags

    # Initial list
    factor_group_rows = []
    factor_group_row_text = []
    factor_group_row_tags = []

    # VBox container that will hold all rows
    rows_container = widgets.VBox(factor_group_rows)

    # Button to add rows
    def add_new_factor_group_row(btn):
        new_row, new_row_text_box, new_row_selected_tags = factor_group_row(allowed_tags)
        factor_group_rows.append(new_row)
        factor_group_row_text.append(new_row_text_box)
        factor_group_row_tags.append(new_row_selected_tags)
        rows_container.children = factor_group_rows 

    add_additional_button = widgets.Button(
        description='Add new grouping',
        button_style='',
        tooltip='Adds a new row',
        icon='plus'
    )
    add_additional_button.on_click(add_new_factor_group_row)

    # Full layout
    inner_accordion = widgets.VBox([
        rows_container,
        add_additional_button
    ])

    return inner_accordion, factor_group_row_text, factor_group_row_tags


In [ ]:
children=[generate_inner_accordion(unique_values_for_each_col[i]) for i in col_selection_checkbox_dict if col_selection_checkbox_dict[i].value]
children_text_boxes = [i[1] for i in children]
children_widgets = [i[0] for i in children]
children_tags = [i[2] for i in children]
accordion = widgets.Accordion(children_widgets, titles=([i for i in col_selection_checkbox_dict if col_selection_checkbox_dict[i].value]))


In [ ]:
accordion

In [ ]:
titles=([i for i in col_selection_checkbox_dict if col_selection_checkbox_dict[i].value])

In [ ]:
ehr_df.columns

In [ ]:
factor_group_dict = {}

factor_group_dict['age'] = {
    'Age <= 65': list(range(0, 66)), 
    'Age > 65': list(range(66, 200))
}

for idx, title in enumerate(titles):
    factor_group_dict[title] = {i.value:j.value for i, j in zip(children_text_boxes[idx], children_tags[idx])}


for k, v in factor_group_dict.items():
    validate_groups(k, v)



# Logistic Regression

In [ ]:

import statsmodels.api as sm
import statsmodels.formula.api as smf

In [ ]:
X = ehr_df.drop(columns=['date_of_birth', 'ethnicity_concept_id', 'sex_at_birth_concept_id', 'has_ehr_data'])


In [ ]:
def factorize_and_group_v2(series, category_mapping):
    # Create a mapping dictionary to group categories
    group_mapping = {}
    for group, categories in category_mapping.items():
        for category in categories:
            group_mapping[category] = group

    # Factorize the original Series based on the group_mapping
    factorized_series = series.map(group_mapping)
    
    # Fill NaN values with the original category values
    factorized_series = factorized_series.fillna(series)

    return factorized_series

# Example usage:
# test_example = pd.Series(["Apple", "Apple", "Banana", "Banana", "Banana", "Banana", "Carrot", "Durian", "Avocado", "Tomato"])
# category_mapping = {
#     "Banana and Carrot": ["Banana", "Carrot"],
#     "Apple": ["Apple"],
#     "Fruit?": ["Tomato"]
# }

# output = factorize_and_group_v2(test_example, category_mapping)
# print(output)

In [ ]:
categorical_col_df_with_y = X[['person_id']]
factor_factorized_dict = {}

for k, v in factor_group_dict.items():
    print(f"Processing for: {k}")
    col = X[k]
    if k == 'age':
        factor_factorized_dict[k] = factorize_and_group_v2(col, v)
    else:
        factor_factorized_dict[k] = factorize_and_group_v2(col, v)
    categorical_col_df_with_y.loc[:, [k]] = factor_factorized_dict[k]

In [ ]:
def compute_logreg_stats(col_name, cat_col_groups, reference_col, _X, _y):
    _y = _y.apply(lambda x: 0 if x[-2:] == 'No' else 1)
    grouped_categorical_col = _X.copy()
    cat_col = _X[col_name]
    grouped_categorical_col[col_name] = factorize_and_group_v2(cat_col, cat_col_groups)
    grouped_categorical_col['condition_of_interest'] = _y
#     grouped_categorical_col['categorical_age'] = grouped_categorical_col['categorical_age'].astype(int) 
    res2 = smf.logit(f"condition_of_interest ~ C({col_name}, Treatment(reference='{reference_col}'))", data=grouped_categorical_col).fit()
#     display(res2.summary())
    odds_ratios = pd.DataFrame(
        {
            "OR": res2.params,
            "Lower CI": res2.conf_int()[0],
            "Upper CI": res2.conf_int()[1],
        }
    )
    odds_ratios = np.exp(odds_ratios)
    print("Odds Ratios")
    pd.set_option('display.precision', 2)
    display(odds_ratios)
    return grouped_categorical_col[col_name], odds_ratios

def get_counts_stats(col_name, cat_col_groups, reference_col, _X, _y):
    _y = _y.apply(lambda x: 0 if x[-2:] == 'No' else 1)
    grouped_categorical_col = _X.copy()
    cat_col = _X[col_name]
    grouped_categorical_col[col_name] = factorize_and_group_v2(cat_col, cat_col_groups)
    display(grouped_categorical_col.groupby(col_name).count()['person_id'])
    grouped_categorical_col['condition_of_interest'] = y
#     grouped_categorical_col['categorical_age'] = grouped_categorical_col['categorical_age'].astype(int) 
    return grouped_categorical_col[col_name], odds_ratios

def compute_logreg_stats_age_only(col_name, cat_col_groups, reference_col, _X, _y):
    _y = _y.apply(lambda x: 0 if x[-2:] == 'No' else 1)
    grouped_categorical_col = _X.copy()
    cat_col = _X[col_name]
    grouped_categorical_col[col_name] = factorize_and_group_v2(cat_col, cat_col_groups)
    grouped_categorical_col['condition_of_interest'] = _y
    # Age removed below because singleton matrix error when using age twice 
    res2 = smf.logit(f"condition_of_interest ~ C({col_name}, Treatment(reference='{reference_col}'))", data=grouped_categorical_col).fit()
    odds_ratios = pd.DataFrame(
        {
            "OR": res2.params,
            "Lower CI": res2.conf_int()[0],
            "Upper CI": res2.conf_int()[1],
        }
    )
    odds_ratios = np.exp(odds_ratios)
    print("Odds Ratios")
#     pd.set_option('display.precision', 2)
    pd.set_option('display.float_format', lambda x: '%.2f' % x)
    display(odds_ratios)
    return grouped_categorical_col[col_name], odds_ratios

In [ ]:
categorical_col_df_with_y.columns


In [ ]:
# # remove unknown race 
# categorical_col_df_with_y = categorical_col_df_with_y[
#     categorical_col_df_with_y['race'].isin(list(factor_group_dict['race'].keys())[:2])
# ]

In [ ]:
# list(factor_group_dict['race'].keys())[:2]


In [ ]:
# factor_factorized_dict = {}
# factor_odds_ratios_dict = {}
# for k, v in factor_group_dict.items():
#     print(f"Processing for: {k}")
#     if k == 'age':
#         factor_factorized_dict[k], factor_odds_ratios_dict[k] = compute_logreg_stats_age_only(k, v, list(v.keys())[0], categorical_col_df_with_y.copy(), categorical_col_df_with_y[k])
#     else:
#         factor_factorized_dict[k], factor_odds_ratios_dict[k] = compute_logreg_stats(k, v, list(v.keys())[0], categorical_col_df_with_y.copy(), categorical_col_df_with_y[k])
# #     categorical_col_df_with_y.loc[:, [k]] = factor_factorized_dict[k]

In [ ]:
def compute_multivariate_logreg_stats(col_names, _cat_col_df_with_y, _all_references, _target_variable):
    
    
    grouped_categorical_col = _cat_col_df_with_y.copy()
#     res2 = smf.logit(f"psoriasis ~ age_above_65 + C({col_name}, Treatment(reference=0))", data=grouped_categorical_col).fit()
    formula_logit = " + ".join([f"C({i}, Treatment(reference='{j}'))" for i, j in zip(col_names, _all_references)])
    formula_logit = f"{_target_variable} ~ {formula_logit}"
#     print(f"Using formula {'\n'.join(formula_logit.split('+'))}")
    for i in formula_logit.split("+"):
        print(i)
    res2 = smf.logit(formula_logit, data=grouped_categorical_col, missing='drop').fit(method='bfgs')
    
#     display(res2.summary())
    odds_ratios = pd.DataFrame(
        {
            "OR": res2.params,
            "Lower CI": res2.conf_int()[0],
            "Upper CI": res2.conf_int()[1],
        }
    )
    odds_ratios = np.exp(odds_ratios)
    print("Odds Ratios")
    pd.set_option('display.precision', 2)
    display(odds_ratios.round(2))
    return odds_ratios

In [ ]:
def pretty_print_or(ugly_df):
    beautiful_odds_ratio = ugly_df.reset_index()
    beautiful_odds_ratio.iloc[0,0] = "C(Intercept, Treatment(reference='Intercept'))[T.Intercept]"
    category_name = beautiful_odds_ratio['index'].apply(lambda x: x.split(', Treatment(reference=')[0][2:])
    category_value = beautiful_odds_ratio['index'].apply(lambda x: x.split('))[')[-1][2:-1])
    refernece_variable = beautiful_odds_ratio['index'].apply(lambda x: x.split(", Treatment(reference=")[1].split("))[")[0])
    beautiful_odds_ratio = beautiful_odds_ratio.drop(columns=['index'])
    beautiful_odds_ratio['Category Name'] = category_name
    beautiful_odds_ratio['Reference'] = refernece_variable
    beautiful_odds_ratio['Category Value'] = category_value
#     beautiful_odds_ratio = beautiful_odds_ratio.set_index(['Category Name','Reference', 'Category Value'])
    return beautiful_odds_ratio

In [ ]:
def pretty_print_or_age_adj(ugly_df):
    beautiful_odds_ratio = ugly_df.reset_index()
    beautiful_odds_ratio.iloc[0,0] = "C(Intercept, Treatment(reference='Intercept'))[T.Intercept]"
    beautiful_odds_ratio.iloc[-1,0] = "C(Age, Treatment(reference='Age'))[T.Age]"
    category_name = beautiful_odds_ratio['index'].apply(lambda x: x.split(', Treatment(reference=')[0][2:])
    category_value = beautiful_odds_ratio['index'].apply(lambda x: x.split('))[')[-1][2:-1])
    refernece_variable = beautiful_odds_ratio['index'].apply(lambda x: x.split(", Treatment(reference=")[1].split("))[")[0])
    beautiful_odds_ratio = beautiful_odds_ratio.drop(columns=['index'])
    beautiful_odds_ratio['Category Name'] = category_name
    beautiful_odds_ratio['Reference'] = refernece_variable
    beautiful_odds_ratio['Category Value'] = category_value
#     beautiful_odds_ratio = beautiful_odds_ratio.set_index(['Category Name','Reference', 'Category Value'])
    return beautiful_odds_ratio

def compute_logreg_stats_infinity(col_name, reference_col):
    grouped_categorical_col = infinity_fix_attempt1.copy()
    grouped_categorical_col['age_above_65'] = (grouped_categorical_col['age_above_65'] == 'Age > 65').astype(int)
#     Age > 65
    res2 = smf.logit(f"condition_of_interest ~ age_above_65 + C({col_name}, Treatment(reference='{reference_col}'))", data=grouped_categorical_col).fit()
    odds_ratios = pd.DataFrame(
        {
            "OR": res2.params,
            "Lower CI": res2.conf_int()[0],
            "Upper CI": res2.conf_int()[1],
        }
    )
    odds_ratios = np.exp(odds_ratios)
#         f"{odds_ratios[]} ({res2.conf_int()[0]} – {res2.conf_int()[1]})",

    print("Odds Ratios")
    pd.set_option('display.precision', 2)
#     display(odds_ratios)
    return odds_ratios

def compute_logreg_stats_infinity_age_only(col_name, reference_col):
    grouped_categorical_col = infinity_fix_attempt1.copy()
#     grouped_categorical_col['age_above_65'] = (grouped_categorical_col['age_above_65'] == 'Age > 65').astype(int)
#     Age > 65
    res2 = smf.logit(f"condition_of_interest ~ C({col_name}, Treatment(reference='{reference_col}'))", data=grouped_categorical_col).fit()
    odds_ratios = pd.DataFrame(
        {
            "OR": res2.params,
            "Lower CI": res2.conf_int()[0],
            "Upper CI": res2.conf_int()[1],
        }
    )
    odds_ratios = np.exp(odds_ratios)
    print("Odds Ratios")
    pd.set_option('display.precision', 2)
#     display(odds_ratios)
    return odds_ratios

In [ ]:
# age_adjusted_odds_ratios_dfs = []
# for k, v in factor_group_dict.items():
#     print(f"Calculating for {k}")
#     if k == (condition_of_interest.replace(" ", "_")):
#         print(f"SKIPPING {k}")
#         continue
#     if k == 'age_above_65':
#         age_adjusted_odds_ratios_dfs.append(pretty_print_or(compute_logreg_stats_infinity_age_only(k, list(v.keys())[0])))
#     else:
#         try:
#             age_adjusted_odds_ratios_dfs.append(pretty_print_or_age_adj(compute_logreg_stats_infinity(k, list(v.keys())[0])))
#         except np.linalg.LinAlgError as e:
#             print(f"FAILED TO COMPUTE FOR {k}")
# v3 = ''
# for i in age_adjusted_odds_ratios_dfs:
#     display(i)
#     v2 = i[["OR", "Lower CI", "Upper CI"]].apply((lambda x: f"{x[0]:.2f} ({x[1]:.2f} – {x[2]:.2f})"), result_type='reduce', axis=1)
#     for i in v2[1:-1]:
#         v3+=str(i)+"\n"
# print("AGE OR LCI UCI NOT PRINTED!!! PLEASE ADD MANUALLY")
# print(v3)

In [ ]:
# autoimmune_removed_groups = [i for i in factor_group_dict.keys() if i not in [j.replace(' ', '_') for j in autoimmune_conditions.keys()]]
autoimmune_removed_groups = [i for i in factor_group_dict.keys()]
autoimmune_removed_factor_group_dict = {k:v for k,v in factor_group_dict.items() if k in autoimmune_removed_groups}
all_groups = autoimmune_removed_factor_group_dict.values()
all_references = [list(i.keys())[0] for i in all_groups]
all_references

In [ ]:
def compute_MVALR(_target_variable, _X, pos_label, _all_references):
    _X.loc[_X[_target_variable] == pos_label, _target_variable] = 1
    _X.loc[_X[_target_variable] != 1, _target_variable] = 0
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
#     _X.loc[:, _target_variable] = _X[_target_variable].astype(int)
    _X[_target_variable] = _X[_target_variable].astype(int)
    multivariate_odds_ratio = compute_multivariate_logreg_stats(
        [i for i in _X if not(i in [_target_variable, sanitize(_target_variable)])], 
        _X,
        [i for i in _all_references if i != f"{_target_variable}: No"],
        _target_variable
    )
    # pd.set_option('display.precision', 2)
    pd.set_option('display.float_format', '{:.2f}'.format)
    
#     display(multivariate_odds_ratio)
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
    print(f"{'-'*50}")
    return multivariate_odds_ratio

def compute_AALR(_target_variable, _X, pos_label, _all_references):
    _X.loc[_X[_target_variable] == pos_label, _target_variable] = 1
    _X.loc[_X[_target_variable] != 1, _target_variable] = 0
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
#     _X.loc[:, _target_variable] = _X[_target_variable].astype(int)
    _X[_target_variable] = _X[_target_variable].astype(int)
    multivariate_odds_ratio = compute_multivariate_logreg_stats(
        ['age', 'race'], 
        _X,
        ['Age <= 65', 'White'],
        _target_variable
    )
    # pd.set_option('display.precision', 2)
    pd.set_option('display.float_format', '{:.2f}'.format)
    
#     display(multivariate_odds_ratio)
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
    print(f"{'-'*50}")
    return multivariate_odds_ratio

def compute_all_AALR(_target_variable, _X, pos_label, _all_references):
    _X.loc[_X[_target_variable] == pos_label, _target_variable] = 1
    _X.loc[_X[_target_variable] != 1, _target_variable] = 0
    _ar = _all_references.copy()
    multivariate_odds_ratio_df = pd.DataFrame(columns =['index','OR', 'Lower CI', 'Upper CI'])
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
#     _X.loc[:, _target_variable] = _X[_target_variable].astype(int)
    for col in _X.columns:
        if col in ['person_id', 'age', _target_variable]:
            print(f"Skipping for {col} as predictor")
            continue
        else:
            print(f"Processing for {col} as predictor")
            ref_var = _ar.pop(0)
            print(f"Reference = {ref_var}")
            _X[_target_variable] = _X[_target_variable].astype(int)
            multivariate_odds_ratio = compute_multivariate_logreg_stats(
                ['age', col], 
                _X,
                ['Age <= 65', ref_var],
                _target_variable
            )
            print(f"multivariate_odds_ratio cols = {multivariate_odds_ratio.columns}")
#             multivariate_odds_ratio_dict[col] = multivariate_odds_ratio
            multivariate_odds_ratio_df = pd.concat([multivariate_odds_ratio_df, multivariate_odds_ratio.reset_index()])
    # pd.set_option('display.precision', 2)
    pd.set_option('display.float_format', '{:.2f}'.format)
    
#     display(multivariate_odds_ratio)
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
    print(f"{'-'*50}")
    return multivariate_odds_ratio_df

In [ ]:
def compute_AALR_only(_target_variable, _X, pos_label, _all_references):
    
#     compute_multivariate_logreg_stats(col_names, _cat_col_df_with_y, _all_references, _target_variable)
    _X.loc[_X[_target_variable] == pos_label, _target_variable] = 1
    _X.loc[_X[_target_variable] != 1, _target_variable] = 0
    _ar = _all_references.copy()
    multivariate_odds_ratio_df = pd.DataFrame(columns =['index','OR', 'Lower CI', 'Upper CI'])
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
#     _X.loc[:, _target_variable] = _X[_target_variable].astype(int)
#     for col in ['age']:
    col = 'age'
    print(f"Processing for {col} as predictor")
    ref_var = _ar.pop(0)
    print(f"Reference = {ref_var}")
    _X[_target_variable] = _X[_target_variable].astype(int)
    multivariate_odds_ratio = compute_multivariate_logreg_stats(
        ['age', col], 
        _X,
        ['Age <= 65', ref_var],
        _target_variable
    )
    print(f"multivariate_odds_ratio cols = {multivariate_odds_ratio.columns}")
#             multivariate_odds_ratio_dict[col] = multivariate_odds_ratio
#     multivariate_odds_ratio_df = pd.concat([multivariate_odds_ratio_df, multivariate_odds_ratio.reset_index()])
    multivariate_odds_ratio_df = multivariate_odds_ratio.reset_index()
    # pd.set_option('display.precision', 2)
    pd.set_option('display.float_format', '{:.2f}'.format)
    
#     display(multivariate_odds_ratio)
    print("COUNTS:::")
    print(f"POS counts = {len(_X.loc[_X[_target_variable] == 1])}")
    print(f"NEG counts = {len(_X.loc[_X[_target_variable] == 0])}")
    print(f"{'-'*50}")
    return multivariate_odds_ratio_df

In [ ]:
categorical_col_df_with_y


In [ ]:

# !pip install itables
# from itables import init_notebook_mode, show
# init_notebook_mode(all_interactive=True)
# from itables import init_notebook_mode, show

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [ ]:
######## UNCOMMENT CODE IF RUNNING WITHOUT ANY SUBSET #########
condition_only_df = categorical_col_df_with_y
all_references_no_condition = all_references

In [ ]:
# # def compute_MVALR(_target_variable, _X, pos_label, _all_references):
# compute_MVALR('Squamous_cell_carcinoma', categorical_col_df_with_y.drop(columns=['person_id']), 'Squamous_cell_carcinoma: Yes', [i for i in all_references if i != 'Squamous_cell_carcinoma: No'])


In [ ]:
categorical_col_df_with_y.columns


In [ ]:
# ######## UNCOMMENT CODE IF RUNNING A SUBSET #########
# condition_to_subset = 'Kaposis_sarcoma_clinical'
# condition_only_df = categorical_col_df_with_y.copy()
# condition_only_df = condition_only_df.loc[condition_only_df[condition_to_subset] == f'{condition_to_subset}: Yes' , :]
# condition_only_df = condition_only_df.drop(columns=[condition_to_subset])
# all_references_no_condition = [i for i in all_references if i != f'{condition_to_subset}: No']

In [ ]:
# # treatment_df_dict + hiv
# treatment_df_dict.keys()

# immunocompromised_cols = [i for i in condition_only_df.columns if i[:3] == "L01"] + ['L04', 'Human_immunodeficiency_virus_infection']


# condition_only_df[immunocompromised_cols].apply(
#     lambda x: x.apply(
#         lambda y: 1 if y[-2:] == 'es' else 0
#     ).sum() > 0, axis = 1
# )

In [ ]:
# condition_only_df["Immunocompromised"] = condition_only_df[immunocompromised_cols].apply(
#     lambda x: x.apply(
#         lambda y: 1 if y[-2:] == 'es' else 0
#     ).sum() > 0, axis = 1
# )

In [ ]:
# condition_only_df["Immunocompromised"] = condition_only_df["Immunocompromised"].apply(lambda x: "Immunocompromised: Yes" if x else "Immunocompromised: No")


In [ ]:
# condition_only_df["Immunocompromised"].value_counts()


In [ ]:
# condition_only_df = condition_only_df.drop(columns=immunocompromised_cols)


In [ ]:
# condition_only_df.columns


In [ ]:
# all_references_no_condition = [i for i in all_references_no_condition if i.split(':')[0] not in  immunocompromised_cols]


In [ ]:
# for c in prurigo_only_df.columns:
#     if c == 'person_id':
#         continue
#     display(prurigo_only_df.drop(columns=['person_id']).loc[prurigo_only_df['Schizophrenia'] == 'Schizophrenia: Yes', [c]].value_counts())


In [ ]:
# condition_only_df["Hypothyroidism"]

In [ ]:
all_references_no_condition

In [ ]:
condition_only_df.columns

In [ ]:
# condition_only_df["Thyroid_Condition"] = condition_only_df["Thyroid Condition"]

# CHANGE EXPOSURE AND OUTCOME HERE

In [ ]:

# Defaults
target_column = 'Race__What_Race_Ethnicity'
target_positive_value = "White"
target_reference_value = "White"

# Dropdown widgets
col_dropdown = widgets.Dropdown(
    options=condition_only_df.columns.tolist(),
    value=target_column,
    description='Column:'
)

positive_dropdown = widgets.Dropdown(description='Positive:')
reference_dropdown = widgets.Dropdown(description='Reference:')

# Update function
def update_value_dropdowns(change):
    col = change['new']
    vals = sorted(condition_only_df[col].dropna().unique().tolist())
    positive_dropdown.options = vals
    reference_dropdown.options = vals
    # set defaults if exist
    if col == target_column:
        if target_positive_value in vals:
            positive_dropdown.value = target_positive_value
        if target_reference_value in vals:
            reference_dropdown.value = target_reference_value

# Observe changes
col_dropdown.observe(update_value_dropdowns, names='value')

# Initialize with default column
update_value_dropdowns({'new': target_column})

# Display UI
ui = widgets.VBox([
    col_dropdown,
    positive_dropdown,
    reference_dropdown
])
display(ui)

In [ ]:
target_column = col_dropdown.value
target_positive_value = positive_dropdown.value
target_reference_value = reference_dropdown.value

In [ ]:
target_column

In [ ]:
# target_column = "Thyroid_Condition"
# # target_positive_value = f'{target_column}: Yes'
# # target_reference_value = f'{target_column}: No'
# target_positive_value = 'Yes'
# target_reference_value = 'No'
condition_as_outcome_ors = compute_MVALR(target_column, condition_only_df.drop(columns=['person_id']), target_positive_value, [i for i in all_references_no_condition if i != target_reference_value])
condition_as_outcome_ors = condition_as_outcome_ors.reset_index()
condition_as_outcome_ors.columns = ['Exposure', 'OR', 'Lower CI', 'Upper CI']
condition_as_outcome_ors['Outcome'] = target_positive_value
condition_as_outcome_ors = condition_as_outcome_ors[['Outcome', 'Exposure', 'OR', 'Lower CI', 'Upper CI']]

In [ ]:
color_print(f'MVALR with "{target_column}": "{target_positive_value}" as outcome', 'blue')
display(condition_as_outcome_ors)

In [ ]:
condition_as_outcome_ors_age_only = compute_AALR_only(target_column, condition_only_df.drop(columns=['person_id']), target_positive_value, [i for i in all_references_no_condition if i != target_reference_value])
condition_as_outcome_ors_age_only.columns = ['Exposure', 'OR', 'Lower CI', 'Upper CI']
condition_as_outcome_ors_age_only['Outcome'] = target_positive_value
condition_as_outcome_ors_age_only = condition_as_outcome_ors_age_only[['Outcome', 'Exposure', 'OR', 'Lower CI', 'Upper CI']]

In [ ]:
condition_as_outcome_age_adj_ors = compute_all_AALR(target_column, condition_only_df.drop(columns=['person_id']), target_positive_value, [i for i in all_references_no_condition if i not in ['Age <= 65', target_reference_value]])
display(condition_as_outcome_age_adj_ors)
condition_as_outcome_age_adj_ors = condition_as_outcome_age_adj_ors
condition_as_outcome_age_adj_ors.columns = ['Exposure', 'OR', 'Lower CI', 'Upper CI']
condition_as_outcome_age_adj_ors['Outcome'] = target_positive_value
condition_as_outcome_age_adj_ors = condition_as_outcome_age_adj_ors[['Outcome', 'Exposure', 'OR', 'Lower CI', 'Upper CI']]

In [ ]:
pd.set_option('display.max_rows', 500)
color_print(f'AALR with "{target_column}": "{target_positive_value}" as outcome', 'blue')
display(condition_as_outcome_age_adj_ors)

In [ ]:
color_print(f'AALR ONLY "{target_column}": "{target_positive_value}" as outcome', 'blue')
display(condition_as_outcome_ors_age_only)


In [ ]:
# def extract_row_matching(or_df, pattern):
#     or_df = or_df.reset_index()
#     or_df = or_df[or_df['index'].str.contains(pattern)]
#     return or_df

In [ ]:
# mental_comorbidities = [
#     'Major_depressive_disorder', 
#     'Generalized_anxiety_disorder',
#     'Bipolar_disorder',
#     'Insomnia',
#     'Schizophrenia',
#     'Posttraumatic_stress_disorder',
#     'Alcoholism'
# ]
# odds_ratio_dict = {}
# odds_ratio_dict_age_adjusted = {}
# for mc in mental_comorbidities:
#     odds_ratio_dict[mc] = compute_MVALR(mc, prurigo_only_df.drop(columns=['person_id']), f'{mc}: Yes', all_references_no_prurigo)
#     odds_ratio_dict_age_adjusted[mc] = compute_AALR(mc, prurigo_only_df.drop(columns=['person_id']), f'{mc}: Yes', all_references_no_prurigo)

In [ ]:
# MVALR_conditions_df = pd.DataFrame(columns=['Outcome', 'Exposure', 'OR', 'Lower CI', 'Upper CI'])
# # display(Markdown('# Multi-Variable Adjusted ORs'))

# for k in odds_ratio_dict.keys():
#     row_df = extract_row_matching(odds_ratio_dict[k], 'Person')
#     row_df.columns = ['Exposure', 'OR', 'Lower CI', 'Upper CI']
#     row_df['Outcome'] = k
#     row_df = row_df[['Outcome', 'Exposure', 'OR', 'Lower CI', 'Upper CI']]
#     MVALR_conditions_df = pd.concat([MVALR_conditions_df, row_df])

# color_print('Multi-Variable Adjusted ORs', 'blue')
# display(MVALR_conditions_df)

In [ ]:
# odds_ratio_dict_age_adjusted


# age_adjusted_ors_df = pd.DataFrame(columns=['Outcome', 'Exposure', 'OR', 'Lower CI', 'Upper CI'])
# for mc, mc_df in odds_ratio_dict_age_adjusted.items():
#     mc_df_ri = mc_df.reset_index()
#     mc_df_ri.columns = ['Exposure', 'OR', 'Lower CI', 'Upper CI']
#     mc_df_ri['Outcome'] = mc
#     mc_df_ri = mc_df_ri[['Outcome', 'Exposure', 'OR', 'Lower CI', 'Upper CI']]
# #     display(mc_df_ri)
# #     break
#     age_adjusted_ors_df = pd.concat([age_adjusted_ors_df, mc_df_ri])
# color_print('Age Adjusted ORs', 'blue')
# display(age_adjusted_ors_df)

In [ ]:
for k, v in ingredient_df_dict.items():
    treat_df = dataset_person_df[['person_id']].copy()
    v[k.replace(" ", "_").replace("(","").replace(")", "")] = f"{k}: Yes"
    treat_df = treat_df.merge(v, on='person_id', how="left")
    treat_df = treat_df.fillna(f"{k}: No")
    main_df_with[k] = treat_df

In [ ]:
for k, v in ingredient_df_dict.items():
    condition_only_df = condition_only_df.merge(main_df_with[k][['person_id', k.replace(" ", "_").replace("(","").replace(")", "")]], on='person_id', how='left')
    print(final_df.shape)

In [ ]:
# target_column = 'race'
# target_positive_value = 'Person of Color'
# target_reference_value = 'White'

In [ ]:
counts_parent_df = condition_only_df.drop(columns=['person_id'])
# counts_df_records = pd.DataFrame(columns=['factor', 'race', 'counts'])
# display(Markdown('# Counts with percentages'))
color_print('Counts with percentages', 'blue')
for c in counts_parent_df.columns:
    if c == target_column:
        continue
    c_df = counts_parent_df[[target_column, c]].groupby([target_column, c]).value_counts().to_frame()
    sums = c_df.groupby([target_column]).agg('sum')    
    total = c_df['count'].sum()
    c_df['percentage'] = c_df.apply(lambda x:  int(x['count'])*100/int(sums.loc[x.name[0], 'count']), axis=1)
    c_df['percentage'] = c_df['percentage'].apply(lambda x: f"{x:.2f} %")
    display(c_df)
#     break

In [ ]:
pd.set_option('display.max_columns', 20)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.min_rows', 1000)

In [ ]:
counts_parent_df = condition_only_df.drop(columns=['person_id'])
counts_df_final = pd.DataFrame(columns=['factor', target_column, 'count', 'percentage'])
for c in counts_parent_df.columns:
    if c == target_column:
        continue
    c_df = counts_parent_df[[target_column, c]].groupby([target_column, c]).value_counts().to_frame()
    sums = c_df.groupby([target_column]).agg('sum')    
    total = c_df['count'].sum()
    c_df['percentage'] = c_df.apply(lambda x:  int(x['count'])*100/int(sums.loc[x.name[0], 'count']), axis=1)
    c_df['percentage'] = c_df['percentage'].apply(lambda x: f"{x:.2f} %")
    c_df = c_df.reset_index()
    c_df.columns = ['factor', target_column, 'count', 'percentage']
    counts_df_final = pd.concat([counts_df_final, c_df], axis=0)

# display(Markdown('# Counts with percentages (easy to copy)'))
color_print('Counts with percentages (easy to copy)', 'blue')
display(counts_df_final)

In [ ]:
# color_print(f'{target_column} Breakdown', 'blue')
# condition_only_df.merge(X[['person_id', target_column]], how='left', on='person_id')[f'{target_column}_y'].value_counts().to_frame().reset_index()


# Outputs Summary

In [ ]:
color_print(f'MVALR with "{target_column}": "{target_positive_value}" as outcome', 'blue')
display(condition_as_outcome_ors)

color_print(f'AALR with "{target_column}": "{target_positive_value}" as outcome', 'blue')
display(condition_as_outcome_age_adj_ors)

color_print(f'AALR ONLY "{target_column}": "{target_positive_value}" as outcome', 'blue')
display(condition_as_outcome_ors_age_only)

# color_print('Multi-Variable Adjusted ORs', 'blue')
# display(MVALR_conditions_df)

# color_print('Age Adjusted ORs', 'blue')
# display(age_adjusted_ors_df)

color_print('Counts with percentages (easy to copy)', 'blue')
display(counts_df_final)

color_print('Race Breakdown', 'blue')
condition_only_df.merge(condition_only_df[['person_id', target_column]], how='left', on='person_id')[f'{target_column}_y'].value_counts().to_frame().reset_index()


In [ ]:
counts_parent_df = condition_only_df.drop(columns=['person_id'])
counts_df_final = pd.DataFrame(columns=['factor', target_column, 'count', 'percentage'])
for c in counts_parent_df.columns:
    if c == target_column:
        continue
    c_df = counts_parent_df[[target_column, c]].groupby([target_column, c]).value_counts().to_frame()
    sums = c_df.groupby([target_column]).agg('sum')    
    total = c_df['count'].sum()
    c_df['percentage'] = c_df.apply(lambda x:  int(x['count'])*100/int(sums.loc[x.name[0], 'count']), axis=1)
    c_df['percentage'] = c_df['percentage'].apply(lambda x: f"{x:.2f} %")
    c_df = c_df.reset_index()
    c_df.columns = ['factor', target_column, 'count', 'percentage']
    counts_df_final = pd.concat([counts_df_final, c_df], axis=0)

# display(Markdown('# Counts with percentages (easy to copy)'))
color_print('Counts with percentages (easy to copy)', 'blue')
display(counts_df_final)

In [ ]:
counts_no_hiv = counts_parent_df.copy()
counts_no_hiv = counts_no_hiv[counts_no_hiv["Human_immunodeficiency_virus_infection"] != "Human_immunodeficiency_virus_infection: Yes"]
counts_no_hiv.shape

In [ ]:
counts_parent_df = counts_no_hiv.copy()

l01_cols = [i for i in counts_parent_df.columns if i[:3] == "L01"]
l01_cols.append('L04')

for c in l01_cols:
    counts_parent_df[c] = counts_parent_df[c].apply(lambda x: True if (x[-3:] == "Yes") else False)

counts_parent_df['L01_L04'] = counts_parent_df[l01_cols].any(axis=1)
counts_parent_df['L01_L04'] = counts_parent_df['L01_L04'].apply(lambda x: 'L01_L04: Yes' if x else 'L01_L04: No')
counts_parent_df = counts_parent_df.drop(columns=l01_cols)




target_column = 'L01_L04'
target_positive_value = f'{target_column}: Yes'
target_reference_value = f'{target_column}: No'

counts_df_final = pd.DataFrame(columns=['factor', target_column, 'count', 'percentage'])

for c in counts_parent_df.columns:
    if c == target_column:
        continue
    c_df = counts_parent_df[[target_column, c]].groupby([target_column, c]).value_counts().to_frame()
    sums = c_df.groupby([target_column]).agg('sum')    
    total = c_df['count'].sum()
    c_df['percentage'] = c_df.apply(lambda x:  int(x['count'])*100/int(sums.loc[x.name[0], 'count']), axis=1)
    c_df['percentage'] = c_df['percentage'].apply(lambda x: f"{x:.2f} %")
    c_df = c_df.reset_index()
    c_df.columns = ['factor', target_column, 'count', 'percentage']
    counts_df_final = pd.concat([counts_df_final, c_df], axis=0)

# display(Markdown('# Counts with percentages (easy to copy)'))
color_print('Counts (no HIV) with percentages (easy to copy)', 'blue')
display(counts_df_final)

In [ ]:
color_print(f'{target_column} Breakdown', 'blue')
print(f"Number of subjects with L01_L04: Yes = {counts_parent_df['L01_L04'].apply(lambda x: 1 if x == 'L01_L04: Yes' else 0).sum()}")
print(f"Number of subjects with L01_L04: No = {counts_parent_df['L01_L04'].apply(lambda x: 1 if x == 'L01_L04: No' else 0).sum()}")


In [ ]:
factor_group_dict = {}

factor_group_dict['age'] = {
    'Age <= 65': list(range(0, 66)), 
    'Age > 65': list(range(66, 200))
}

for idx, title in enumerate(titles):
    factor_group_dict[title] = {i.value:j.value for i, j in zip(children_text_boxes[idx], children_tags[idx])}


# # factor_group_dict['race'] = {
# #     'White': ['What Race Ethnicity: White'], 
# #     'Black or African American': ['What Race Ethnicity: Black'], 
# #     'Hispanic': ['What Race Ethnicity: Hispanic'], 
# #     'Asian': ['What Race Ethnicity: Asian'],
# #     'More than... + Another single... + None of...': [
# #         'What Race Ethnicity: Race Ethnicity None Of These',
# #         'What Race Ethnicity: NHPI',
# #         'What Race Ethnicity: MENA',
# #         'Race: Multiselect'
# #     ],
# #     'Skip + prefer not...': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'Race: Unknown']
# # }

# factor_group_dict['race'] = {
#     'White': ['What Race Ethnicity: White'],
#     'Black': ['What Race Ethnicity: Black'],
#     'Asian': ['What Race Ethnicity: Asian'],
#     'Hispanic': ['What Race Ethnicity: Hispanic'],
#     'Other': [
#         'What Race Ethnicity: Race Ethnicity None Of These',
#         'What Race Ethnicity: NHPI',
#         'What Race Ethnicity: MENA',
#         'What Race Ethnicity: AIAN',
#         'Race: Multiselect',
        
#     ],
#     'Skip + prefer not...': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'Race: Unknown']
# }


# factor_group_dict['ethnicity'] = {
#     'Not Hispanic or Latino': ['Not Hispanic or Latino'], 
#     'Hispanic or Latino': ['Hispanic or Latino'], 
#     'Ethnicity None Of These': ['What Race Ethnicity: Race Ethnicity None Of These'], 
#     'Prefer not... + Skip + No matching...': ['PMI: Prefer Not To Answer', 'PMI: Skip', 'No matching concept']
# }

# ################################
# factor_group_dict['gender_identity'] = {
#     'Gender Identity: Man': ['Gender Identity: Man'], 
#     'Gender Identity: Woman': ['Gender Identity: Woman'],
#     'Gender Identity: Transgender': ['Gender Identity: Transgender'],
#     'Gender Other': [ 
#              'Gender Identity: Non Binary', 
#              'Gender Identity: Additional Options', 
#             'Gender Identity: Multiselect'
#             ],
#     'Prefer not... + Skip + unknown + no match': [
#         'PMI: Skip',
#         'PMI: Prefer Not To Answer',
#         'Gender Identity: Unknown',
#             'No matching concept',
#         'Not man only, not woman only, prefer not to answer, or skipped'
        
#     ]
# }

# factor_group_dict['sexual_orientation'] = {
#     'Straight': ['Sexual Orientation: Straight'],
#     'Lesbian': ['Sexual Orientation: Lesbian'],
#     'Gay': ['Sexual Orientation: Gay'],
#     'Bisexual': ['Sexual Orientation: Bisexual'],
#     'multi_select': ['Sexual Orientation: Multiselect', 'None + sexual_orientation_other multiselect'],
# #     'None of the above': ['Sexual Orientation: None'],
# #     'None + Skip': ['None + PMI: Skip'],
#     'None + Something Else' :['None + Sexuality Closer Description: Something Else'] ,
#     'None + Dont Know' :['None + Sexuality Closer Description: Dont Know'] ,
#     'None + Poly Omni Sapio Pansexual' :['None + Sexuality Closer Description: Poly Omni Sapio Pansexual'], 
#     'None + No Labels' :['None + Sexuality Closer Description: No Labels'] ,
# #     'None + No Sexuality' :['None + Sexuality Closer Description: No Sexuality'] ,
#     'None + Asexual' :['None + Sexuality Closer Description: Asexual'] ,
# #     'None + Two Spirit' :['None + Sexuality Closer Description: Two Spirit'] ,
#     'None + Mostly Straight' :['None + Sexuality Closer Description: Mostly Straight'] ,
# #     'None + Not Figured Out' :['None + Sexuality Closer Description: Not Figured Out'] ,
# #     'None + Queer': ['None + Sexuality Closer Description: Queer'] ,
#     'Other more specific': [
#         'None + Sexuality Closer Description: Queer',
#         'None + Sexuality Closer Description: Not Figured Out',
#         'None + Sexuality Closer Description: No Sexuality',
#         'None + Sexuality Closer Description: Two Spirit'
        
#     ],
# #     'None + multiselect': ['None + sexual_orientation_other multiselect'] ,
# #     'None + unknown': ['None + sexual_orientation_other unknown'] ,
    
#     'Prefer not + skip + unknown + (None + unknown)': [
#         'PMI: Skip', 
#         'PMI: Prefer Not To Answer', 
#         'Sexual Orientation: Unknown', 
#         'Sexual Orientation: None',
#         'None + sexual_orientation_other unknown',
#         'None + PMI: Skip',
#         'Non-straight orientation, prefer not to answer, or skipped'
#     ], 
# }


# ################################
# factor_group_dict['sex_at_birth'] = {
#     'Male': ['Male'],
#     'Female': ['Female'],
#     'Intersex + None + No match + unknown': [
#         'Intersex', 
#         'None', 
#         'No matching concept', 
#         'PMI: Skip', 
#         'I prefer not to answer', 
#         'sex_at_birth unknown',
#         'Sex At Birth: Sex At Birth None Of These',
#         'Not male, not female, prefer not to answer, or skipped'
#     ],
    
# }

# factor_group_dict['education_level'] = {
#     'College graduate or advanced degree': [
#         'College graduate or advanced degree',
#         'Highest Grade: College Graduate',
#         'Highest Grade: Advanced Degree'
#     ],
#     'Grade 12/GED + College 1-3 year': [
#         'Highest Grade: Twelve Or GED', 
#         'Highest Grade: College One to Three'
#     ],
#     'Less than high school': [
#         'Less than a high school degree or equivalent',
#         'Highest Grade: Five Through Eight',
#         'Highest Grade: Nine Through Eleven',
#         'Highest Grade: Never Attended',
#         'Highest Grade: One Through Four'
#     ],
#     'Skip + Prefer not... + unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'Education Level: Unknown']
# }


# factor_group_dict['low_income'] = {
#     'High income': [
#         'Annual Income: 25k 35k',
#         'Annual Income: 35k 50k',
#         'Annual Income: 50k 75k',
#         'Annual Income: 75k 100k',
#         'Annual Income: 100k 150k',
#         'Annual Income: 150k 200k',
#         'Annual Income: more 200k'
#     ], #above 25k
#     'Low income': [
#         'Annual Income: less 10k',
#         'Annual Income: 10k 25k'
        
#     ], #below 25k
#     'Income Unknown': ['PMI: Skip', 'Low Income: Unknown', 'PMI: Prefer Not To Answer']
# }


# # factor_group_dict['has_disability'] = {
# #     'No disability': ['No disability'],
# #     'Disabled': ['Disabled'],
# #     'Disability Unknown': ['Disability Unknown']
# # }

# ##### UNCOMMENT TO ADD DISABILITY #######
# disability_list = ['deaf', 'blind', 'difficulty_concentrating']
# disability_survey_pattern = ['Deaf', 'Blind']
# for _disability, pattern in zip(disability_list, disability_survey_pattern):
#     disability = ' '.join([i for i in _disability.split("_")]).capitalize()
#     factor_group_dict[_disability] = {
#         f'{disability}: No': [f'{pattern}: No'],
#         f'{disability}: Yes': [f'{pattern}: Yes'],
#         f'{disability}: Unknown': [
#             f'{pattern}: Unknown',
#             f'{pattern}: Prefer Not To Answer',
#             'PMI: Skip',
            
#         ]
#     }


# # disability_list2 = ['difficulty_walking_climbing', 'difficulty_dressing_bathing', 'difficulty_errands_alone']
# # disability_survey_pattern2 = ['Walking Climbing', 'Dressing Bathing', 'Errands Alone']
# # for _disability, pattern in zip(disability_list2, disability_survey_pattern2):
# #     disability = ' '.join([i.capitalize() for i in _disability.split("_")])
# #     factor_group_dict[_disability] = {
# #         f'{disability}: No': [f'{pattern}: No'],
# #         f'{disability}: Yes': [f'{pattern}: Yes'],
# #         f'{disability}: Unknown': [
# #             f'Difficulty {pattern}: Unknown',
# #             f'{pattern}: Prefer Not To Answer',
# #             'PMI: Skip',
            
# #         ]
# #     }


# # factor_group_dict['takes_genito_urinary_drug'] = {
# #     'Takes Genito Urinary Drug: No': ['Takes Genito Urinary Drug: No'],
# #     'Takes Genito Urinary Drug: Yes': ['Takes Genito Urinary Drug: Yes']
# # }

# # factor_group_dict['takes_genito_urinary_drug'] = {
# #     'Takes Biologic Drug: No': ['Takes Biologic Drug: No'],
# #     'Takes Biologic Drug: Yes': ['Takes Biologic Drug: Yes']
# # }


# factor_group_dict['has_health_insurance'] = {
#     'Health Insurance: Yes': ['Health Insurance: Yes'],
#     'Health Insurance: No': ['Health Insurance: No'],
#     'Health Insurance: Prefer not... + Skip + Dont Know + unknown': [ 'PMI: Prefer Not To Answer',  'PMI: Skip',  'PMI: Dont Know', 'Has Health Insurance: Unknown']
# }


# factor_group_dict['employment_status'] = {
#     'Employed':[
#         'Employed for wages or self-employed',
#         'Employment Status: Self Employed',
#         'Employment Status: Employed For Wages',
#     ],
#     'Unemployed': [
#         'Not currently employed for wages',
#         'Employment Status: Out Of Work One Or More',
#         'Employment Status: Student',
#         'Employment Status: Unable To Work',
#         'Employment Status: Homemaker',
#         'Employment Status: Out Of Work Less Than One',
#         'Employment Status: Retired',
        
        
#     ],
#     'Employment: Skip + Prefer not... + unknown':[
#         'PMI: Skip', 'PMI: Prefer Not To Answer',
#         'Employment Status: Unknown',
#         'Employment Status: Multiselect'
#     ]
# }


# # factor_group_dict['smoking'] = {
# #     '100 Cigs Lifetime: No': ['100 Cigs Lifetime: No'], 
# #     '100 Cigs Lifetime: Yes': ['100 Cigs Lifetime: Yes'], 
# #     'Smoker Unknown': ['PMI: Skip', 'PMI: Dont Know', 'PMI: Prefer Not To Answer', 'Smoking: Unknown'], 
# # }
# # factor_group_dict['alcohol'] = {
# #     'Alcohol Participant: No': ['Alcohol Participant: No'], 
# #     'Alcohol Participant: Yes': ['Alcohol Participant: Yes'], 
# #     'Alcohol Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'Alcohol: Unknown'], 
# # }
# # factor_group_dict['ecig'] = {
# #     'Electric Smoke Participant: No': ['Electric Smoke Participant: No'], 
# #     'Electric Smoke Participant: Yes': ['Electric Smoke Participant: Yes'], 
# #     'Ecig Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'PMI: Dont Know', 'Ecig: Unknown'], 
# # }
# # factor_group_dict['marijuana'] = {
# #     'Marijuana 3 Month Use: Never': ['Marijuana 3 Month Use: Never'], 
# #     'Marijuana Daily, Weekly, Once, Twice, Monthly': [
# #         'Marijuana 3 Month Use: Weekly',
# #         'Marijuana 3 Month Use: One Or Twice',
# #         'Marijuana 3 Month Use: Daily', 'Marijuana 3 Month Use: Monthly',], 
# #     'Marijuana Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'PMI: Dont Know', 'Marijuana: Unknown'], 
# # }

# # factor_group_dict['Prescription_Opioid'] = {
# #     'Prescription Opioid 3 Month Use: Never': ['Prescription Opioid 3 Month Use: Never'], 
# #     'Prescription Opioid Daily, Weekly, Once, Twice, Monthly': [
# #         'Prescription Opioid 3 Month Use: Weekly',
# #         'Prescription Opioid 3 Month Use: One Or Twice',
# #         'Prescription Opioid 3 Month Use: Daily', 'Prescription Opioid 3 Month Use: Monthly',], 
# #     'Prescription Opioid Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'PMI: Dont Know', 'Prescription_Opioid unknown'], 
# # }
# # factor_group_dict[ 'Prescription_Stimulant'] = {
# #     'Prescription Stimulant 3 Month Use: Never': ['Prescription Stimulant 3 Month Use: Never'], 
# #     'Prescription Stimulant Daily, Weekly, Once, Twice, Monthly': [
# #         'Prescription Stimulant 3 Month Use: Weekly',
# #         'Prescription Stimulant 3 Month Use: One Or Twice',
# #         'Prescription Stimulant 3 Month Use: Daily', 
# #         'Prescription Stimulant 3 Month Use: Monthly',], 
# #     'Prescription Stimulant Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'PMI: Dont Know', 'Prescription_Stimulant unknown'], 
# # }
# # factor_group_dict['Sedative'] = {
# #     'Sedative 3 Month Use: Never': ['Sedative 3 Month Use: Never'], 
# #     'Sedative Daily, Weekly, Once, Twice, Monthly': [
# #         'Sedative 3 Month Use: Weekly',
# #         'Sedative 3 Month Use: Once Or Twice',
# #         'Sedative 3 Month Use: Daily', 
# #         'Sedative 3 Month Use: Monthly',], 
# #     'Sedative Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'PMI: Dont Know', 'Sedative unknown'], 
# # }
# # factor_group_dict[ 'Cocaine'] = {
# #     'Cocaine 3 Month Use: Never': ['Cocaine 3 Month Use: Never'], 
# #     'Cocaine Daily, Weekly, Once, Twice, Monthly': [
# #         'Cocaine 3 Month Use: Weekly',
# #         'Cocaine 3 Month Use: Once Or Twice',
# #         'Cocaine 3 Month Use: Daily', 
# #         'Cocaine 3 Month Use: Monthly',], 
# #     'Cocaine Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'PMI: Dont Know', 'Cocaine unknown'], 
# # }
# # factor_group_dict['Street_Opioid'] = {
# #     'Street Opioid 3 Month Use: Never': ['Street Opioid 3 Month Use: Never'], 
# #     'Street Opioid Daily, Weekly, Once, Twice, Monthly': [
# #         'Street Opioid 3 Month Use: Weekly',
# #         'Street Opioid 3 Month Use: One Or Twice',
# #         'Street Opioid 3 Month Use: Daily', 
# #         'Street Opioid 3 Month Use: Monthly',], 
# #     'Street Opioid Unknown': ['PMI: Skip', 'PMI: Prefer Not To Answer', 'PMI: Dont Know', 'Street_Opioid unknown'], 
# # }


# # factor_group_dict['Hookah'] = {
# #     'Hookah: No': ['Hookah Smoke Participant: No'], 
# #     'Hookah: Yes': ['Hookah Smoke Participant: Yes'], 
# #     'Hookah: Unknown': ['PMI: Dont Know', 'PMI: Skip', 'PMI: Prefer Not To Answer', 'Hookah unknown']
# # }
# # factor_group_dict['Cigar'] = {
# #     'Cigar: No': ['Cigar Smoke Participant: No'], 
# #     'Cigar: Yes': ['Cigar Smoke Participant: Yes'], 
# #     'Cigar: Unknown': ['PMI: Dont Know', 'PMI: Skip', 'PMI: Prefer Not To Answer', 'Cigar unknown']
# # }
# # factor_group_dict['Smokeless_Tobacco'] = {
# #     'Smokeless Tobacco: No': ['Smokeless Tobacco Participant: No'], 
# #     'Smokeless Tobacco: Yes': ['Smokeless Tobacco Participant: Yes'], 
# #     'Smokeless Tobacco: Unknown': ['PMI: Dont Know', 'PMI: Skip', 'PMI: Prefer Not To Answer', 'Smokeless_Tobacco unknown']
# # }


# for k in list(conditions_df_dict.keys()):
#     factor_group_dict[k.replace(" ", "_").replace("(","").replace(")", "")] = {
#         f"{k}: No": [f'{k}: No'],
#         f"{k}: Yes": [f'{k}: Yes'],
#     }
    
# # ######## UNCOMMENT TO ADD TREATMENT GROUPS ###########
# # for k in list(treatment_df_dict.keys()):
# #     factor_group_dict[k.replace(" ", "_").replace("(","").replace(")", "")] = {
# #         f"{k}: No": [f'{k}: No'],
# #         f"{k}: Yes": [f'{k}: Yes'],
# #     }
    
# # factor_group_dict[condition_of_interest.replace(" ", "_")] = {
# #         f"{condition_of_interest}: No": [f'{condition_of_interest}: No'],
# #         f"{condition_of_interest}: Yes": [f'{condition_of_interest}: Yes'],
# #     }

# # factor_group_dict['autoimmune_conditions'] = {
# #     'Autoimmune conditions: No': ['Autoimmune conditions: No'],
# #     'Autoimmune conditions: Yes': ['Autoimmune conditions: Yes']
# # }
    

for k, v in factor_group_dict.items():
    validate_groups(k, v)

# # validate_groups('ethnicity', _ethnicity_groups)
# # validate_groups('gender', _gender_groups)
# # validate_groups('sexual_orientation', _sexual_orientation_groups)
# # validate_groups('education_level', education_groups)
# # validate_groups('is_low_income', income_groups)
# # validate_groups('has_disability', has_disability_groups)
# # validate_groups('has_health_insurance', health_insurance_groups)
# # validate_groups('employment_status', employment_groups)
# # validate_groups('smoking', _smoke_groups)
# # validate_groups('alcohol', _alcohol_groups)
# # validate_groups('ecig', _ecig_groups)
# # validate_groups('marijuana', _marijuana_groups)